In [24]:
class Team:
    players = []
    score = []
    wickets = []
    balls = []
    
    def __init__(self, name, players):
        self.name = name
        self.players = players
    
    def addWicket(self, wicket):
        self.wicket.append(wicket)
    
    def addScore(self, runs):
        self.score.append(runs)
    
    def addBall(self, ball):
        self.balls.append(ball)

In [25]:
class Player:
    score = []
    ballsPlayed = []
    ballsThrown = []
    
    def __init__(self, name, playerType):
        self.name = name
        self.playerType = playerType
        self.status = "not out"
    
    def addScore(self, runs):
        self.score.append(runs)
    
    def addBallPlayed(self, ball):
        self.ballsPlayed.append(ball)
    
    def addBallThrown(self, ball):
        self.ballsThrown.append(ball)
    
    def setStatus(self, status):
        self.status = status

In [26]:
class Score:
    def __init__(self, runs, bowler, batsman, ball):
        self.runs = runs
        self.bowler = bowler
        self.batsman = batsman
        self.ball = ball

In [27]:
class Wicket:
    def __init__(self, ball, bowler, batsman, how):
        self.how = how
        self.bowler = bowler
        self.batsman = batsman
        self.ball = ball

In [28]:
class Ball:
    def __init__(self, runs, bowler, batsman, wicket, ballType):
        self.runs = runs
        self.bowler = bowler
        self.batsman = batsman
        self.ballType = ballType
        self.wicket = wicket

In [97]:
class Match:
    
    extras = ['w','NB']
    validRuns = [1,2,3,4,6,0]
    outs = ['RO','W','C','HW','S']
        
    def __init__(self, team1, team2, ground, overs):
        self.team1 = team1
        self.team2 = team2
        self.overs = overs*6
    
    def toss(self):
        import random
        t = random.randint(0,1)
        if t == 0:
            print(self.team1.name + " won the toss and will be batting first.")
            self.battingTeam = self.team1
            self.bowlingTeam = self.team2
        else:
            print(self.team2.name + " won the toss and will be batting first.")
            self.battingTeam = self.team2
            self.bowlingTeam = self.team1
            
    def getBowlerByName(self, name):
        bowlers = []
        for player in self.bowlingTeam.players:
            if(player.playerType == "Bw" or player.playerType == "A"):
                bowlers.append(player)
        for player in bowlers:
            if(player.name == name):
                return player
    
    def getAllBowlers(self):
        bowlers = []
        for player in self.bowlingTeam.players:
            if(player.playerType == "Bw" or player.playerType == "A"):
                bowlers.append(player.name)
        return bowlers
    
    def getAllBowlersExcept(self, name):
        bowlers = []
        for player in self.bowlingTeam.players:
            if(player.playerType == "Bw" or player.playerType == "A"):
                bowlers.append(player.name)
        bowlers.remove(name)
        return bowlers
    
    def validInput(self, userInput):
        return ((userInput in self.extras) or (userInput in self.validRuns) or (userInput in self.outs))
    
    def getBowler(self, bowlerList):
        while True:
            print("Bowlers list: " + str(bowlerList))
            inputBowler = input("Choose bowler(Name): ")
            if(inputBowler in bowlerList):
                bowler = self.getBowlerByName(inputBowler)
                print("Bowler: " + bowler.name)
                return bowler
            else:
                print("Bowler not in team. Try again")
                continue
    
    def getWicket(self, i, bowler, batsman, ball):
        if(i in self.outs):
            wick = Wicket(ball, bowler, batsman, i)
            return wick
        else:
            return None
    
    def getScore(self, i, bowler, batsman, ball):
        if(i in self.validRuns):
            run = Score(int(i), bowler, batsman, ball)
            return run
        elif(i in self.extras):
            run = Score(1, bowler, batsman, ball)
            return run
        else:
            return None
    
    def getNewBatsman(self):
        playerList = self.battingTeam.players
        for p in playerList:
            if(p.status == "not out"):
                return p
    
    def inningBreak(self):
        print("Inning break.")
        scorex = 0
        for score in self.battingTeam.score:
            if(score is not None):
                print(score.runs)
                scorex = scorex + int(score.runs)
        print("Total score '" + self.battingTeam.name + "': " + str(scorex))
        temp = self.battingTeam
        self.battingTeam = self.bowlingTeam
        self.bowlingTeam = temp
        import time
        time.sleep(200)
        
    def play(self):
        ball = 1
        onStrikeBatsman = self.battingTeam.players[0]
        offStrikeBatsman = self.battingTeam.players[1]
        
        print("Player 1: " + onStrikeBatsman.name)
        print("Player 2: " + offStrikeBatsman.name)
        bowler = self.getBowler(self.getAllBowlers())
        
        while True:
            
            i = input("Ball " + str(ball) + " thrown. Predict: ")
            print("You predicted: " + i)
            
            if(ball%6 == 0):
                print("Over complete. Change Bowler.")
                bowler = self.getBowler(self.getAllBowlersExcept(bowler.name))
            
            score = self.getScore(i, bowler, onStrikeBatsman, ball)
            wicket = self.getWicket(i, bowler, onStrikeBatsman, ball)
            delivery = Ball(score, bowler.name, onStrikeBatsman.name, wicket,'')
            
            onStrikeBatsman.addScore(score)
            onStrikeBatsman.addBallPlayed(delivery)
            bowler.addBallThrown(delivery)
            self.battingTeam.addScore(score)

            ball += 1
            
            if(wicket is None):
                onStrikeBatsman.setStatus("not out")
            else:
                print("Thats out.")
                onStrikeBatsman.setStatus("out")
                onStrikeBatsman = self.getNewBatsman()
                if(onStrikeBatsman is None):
                    print("'" + self.battingTeam.name + "' is Allout.")
                    self.inningBreak()
                    ball = 0
                    continue
            
            if(i == 1 or i == 3):
                temp = onStrikeBatsman
                onStrikeBatsman = offStrikeBatsman
                offStrikeBatsman = temp
                
            if(ball == self.overs):
                self.inningBreak()
                ball = 0
                continue

In [98]:
indPlayersDict = {"Rohit": "B", "Virat": "B", "Pandya": "A", "Shami": "Bw", "Bumrah": "Bw"}
pakPlayersDict = {"Babar": "B", "Inzi": "B", "Afridi": "A", "Razzak": "Bw", "Imran": "Bw"}
indPlayers = []
pakPlayers = []
for player in indPlayersDict.keys():
    indPlayers.append(Player(player,indPlayersDict[player]))
    
for player in pakPlayersDict.keys():
    pakPlayers.append(Player(player,pakPlayersDict[player]))

In [99]:
ind = Team("India", indPlayers)
pak = Team("Pakistan", pakPlayers)

In [100]:
match = Match(ind, pak, "chennai", 1)

In [101]:
match.toss()

India won the toss and will be batting first.


In [ ]:
match.play()

Player 1: Rohit
Player 2: Virat
Bowlers list: ['Afridi', 'Razzak', 'Imran']
Choose bowler(Name): Razzak
Bowler: Razzak
Ball 1 thrown. Predict: W
You predicted: W
Thats out.
Ball 2 thrown. Predict: 1
You predicted: 1
Ball 3 thrown. Predict: 2
You predicted: 2
Ball 4 thrown. Predict: 1
You predicted: 1
Ball 5 thrown. Predict: 1
You predicted: 1
Inning break.
Total score 'India': 0


In [71]:
%debug

> <ipython-input-65-07aec37ae8f5>(98)inningBreak()
     96         self.bowlingTeam = temp
     97         import time
---> 98         time.sleep(200)
     99 
    100     def play(self):

ipdb> self.getNewBatsman()
ipdb> 
ipdb> self.getNewBatsman().name
*** AttributeError: 'NoneType' object has no attribute 'name'
ipdb> for p in battingTeam.players print(p.name)
*** SyntaxError: invalid syntax
ipdb> for p in battingTeam.players: print(p.name)
*** NameError: name 'battingTeam' is not defined
ipdb> for p in self.battingTeam.players: print(p.name)
Rohit
Virat
Pandya
Shami
Bumrah
ipdb> for p in battingTeam.players: print(p.status)
*** NameError: name 'battingTeam' is not defined
ipdb> for p in self.battingTeam.players: print(p.status)
not out
not out
not out
not out
not out
--KeyboardInterrupt--
ipdb> n
